# CNN Training with Fashion Data set - Pytorch A - Z

In [49]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as tranforms

torch.set_printoptions(linewidth=120)  # Displa optin for output
torch.set_grad_enabled(True) # Already on by default

In [50]:
print(torch.__version__)
print(torchvision.__version__)

1.1.0
0.3.0


In [0]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

## Fully Connected CNN

In [0]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        # (1) input layer
        t = t
        
        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)
        
        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)
        
        # (6) output layer
        t = self.out(t)
        #t = F.softmax(t, dim=1)
        
        return t

## Load data set 

In [0]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=tranforms.Compose([
        tranforms.ToTensor()
    ])
)

## Training with a Single batch

In [54]:
network = Network()

train_loader = torch.utils.data.DataLoader(
    train_set
    ,batch_size=100
)
optimizer = optim.Adam(network.parameters(), lr=0.01)

batch = next(iter(train_loader)) # Get Batch
images, labels = batch 

preds = network(images) # Pass Batch
loss = F.cross_entropy(preds, labels) # Calculate Loss

loss.backward() # Calculate Gradients
optimizer.step() # Update Weights


print('loss 1:', loss.item())
preds = network(images)
loss = F.cross_entropy(preds, labels)
print('loss 2:', loss.item())

loss 1: 2.292600631713867
loss 2: 2.2792670726776123


## Training with multitle ephochs : The Complet etraining loop

In [55]:
network = Network()

train_loader = torch.utils.data.DataLoader(
    train_set
    ,batch_size=100
    ,shuffle=True
)
optimizer = optim.Adam(network.parameters(), lr=0.01)


for epoch in range(10):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        images, labels = batch 

        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )

epoch 0 total_correct: 47367 loss: 331.27733567357063
epoch 1 total_correct: 51486 loss: 227.72206424176693
epoch 2 total_correct: 52201 loss: 208.65555331110954
epoch 3 total_correct: 52579 loss: 200.81143063306808
epoch 4 total_correct: 52835 loss: 193.1907076537609
epoch 5 total_correct: 52968 loss: 191.2430875748396
epoch 6 total_correct: 53133 loss: 184.7395621612668
epoch 7 total_correct: 53249 loss: 183.85396520793438
epoch 8 total_correct: 53313 loss: 180.6353891044855
epoch 9 total_correct: 53436 loss: 178.67326836287975


In [56]:
total_correct / len(train_set)

0.8906